Referensi:
- https://colab.research.google.com/github/ShawonAshraf/nlu-jointbert-dl2021/blob/main/notebooks/nlu_jointbert_dl21.ipynb

Note: sesuaikan semua `/path/to/` menuju directory yang sesuai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Installs

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 29.1 MB/s 
     |████████████████████████████████| 596 kB 70.1 MB/s 
     |████████████████████████████████| 6.6 MB 62.1 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Imports

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline
figsize = (15, 10)

In [ ]:
df_atis_test = pd.read_csv('https://raw.githubusercontent.com/moore3930/SlotRefine/main/data/atis/test/data', header=None, sep="\t", names=["data", "encoding", "ground_truth"])
df_atis_train = pd.read_csv('https://raw.githubusercontent.com/moore3930/SlotRefine/main/data/atis/train/data', header=None, sep="\t", names=["data", "encoding", "ground_truth"])
df_atis_validation = pd.read_csv('https://raw.githubusercontent.com/moore3930/SlotRefine/main/data/atis/valid/data', header=None, sep="\t", names=["data", "encoding", "ground_truth"])

df_snips_test = pd.read_csv('https://raw.githubusercontent.com/moore3930/SlotRefine/main/data/snips/test/data', header=None, sep="\t", names=["data", "encoding", "ground_truth"])
df_snips_train = pd.read_csv('https://raw.githubusercontent.com/moore3930/SlotRefine/main/data/snips/train/data', header=None, sep="\t", names=["data", "encoding", "ground_truth"])
df_snips_validation = pd.read_csv('https://raw.githubusercontent.com/moore3930/SlotRefine/main/data/snips/valid/data', header=None, sep="\t", names=["data", "encoding", "ground_truth"])

In [ ]:
def create_O_slot(text):
  temp = "O"
  for i in range(1,len(text.split(" "))):
    temp += " O"
  return temp

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer

model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

# Tokenize

In [ ]:
texts = [d for d in df_atis_train.data]
slots = [d for d in df_atis_train.encoding]
all_O = []
for i in slots:
  all_O.append(create_O_slot(i))
# all_O

In [ ]:
def encode_texts(tokenizer, texts, slots, max_length = None, padding = True):
    return tokenizer(texts,slots, padding= padding,truncation=True, max_length = max_length, return_tensors="tf")
max_token_length = encode_texts(tokenizer, texts, slots)["input_ids"].shape[1]
tds = encode_texts(tokenizer, texts, all_O, max_token_length, "max_length")
tds.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
{"input_ids": tds["input_ids"], "token_type_ids": tds["token_type_ids"],  "attention_mask": tds["attention_mask"]}

{'attention_mask': <tf.Tensor: shape=(4478, 252), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_ids': <tf.Tensor: shape=(4478, 252), dtype=int32, numpy=
 array([[ 101,  178, 1328, ...,    0,    0,    0],
        [ 101, 1668, 3868, ...,    0,    0,    0],
        [ 101, 1437, 1143, ...,    0,    0,    0],
        ...,
        [ 101, 1587, 1143, ...,    0,    0,    0],
        [ 101,  178,  112, ...,    0,    0,    0],
        [ 101, 1110, 1175, ...,    0,    0,    0]], dtype=int32)>,
 'token_type_ids': <tf.Tensor: shape=(4478, 252), dtype=int32, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>}

In [ ]:
encoded_texts = tds

# Mapping slots and intents

In [ ]:
intents = [d for d in df_atis_train.ground_truth]
intent_names = list(set(intents))
intent_names.sort()

In [ ]:
intent_map = dict() # index -> intent
for idx, ui in enumerate(intent_names):
    intent_map[ui] = idx
intent_map

{'atis_abbreviation': 0,
 'atis_aircraft': 1,
 'atis_aircraft#atis_flight#atis_flight_no': 2,
 'atis_airfare': 3,
 'atis_airline': 4,
 'atis_airline#atis_flight_no': 5,
 'atis_airport': 6,
 'atis_capacity': 7,
 'atis_cheapest': 8,
 'atis_city': 9,
 'atis_distance': 10,
 'atis_flight': 11,
 'atis_flight#atis_airfare': 12,
 'atis_flight_no': 13,
 'atis_flight_time': 14,
 'atis_ground_fare': 15,
 'atis_ground_service': 16,
 'atis_ground_service#atis_ground_fare': 17,
 'atis_meal': 18,
 'atis_quantity': 19,
 'atis_restriction': 20}

In [ ]:
# map to train_data values
def encode_intents(intents, intent_map):
    encoded = []
    for i in intents:
        encoded.append(intent_map[i])
    # convert to tf tensor
    return tf.convert_to_tensor(encoded, dtype="int32")

encoded_intents = encode_intents(intents, intent_map)

In [ ]:
# encode slots
slot_list = [x.split(" ") for x in df_atis_train.encoding]
slot_names = set()
for td in slot_list:
    for slot in td:
        slot_names.add(slot)
slot_names = list(slot_names)
slot_names.sort()
slot_names.insert(0, "<PAD>")

In [ ]:
slot_map = dict() # slot -> index
for idx, us in enumerate(slot_names):
    slot_map[us] = idx

In [ ]:
# gets slot name from its values
def get_slot_from_word(word, text, slot):
    text = text.split()
    for i in range(len(text)):
        if word == text[i]:
            return slot[i]
    return None

# print(df_atis_train.data[0])
# print(df_atis_train.encoding[0])
# print("slot_name for google is : ", get_slot_from_word("google", df_snips_train.data[0], slot_list[0]))

In [ ]:
import numpy as np

# find the max encoded test length
# tokenizer pads all texts to same length anyway so
# just get the length of the first one's input_ids
max_len = len(encoded_texts["input_ids"][0])

def encode_slots(all_slots, all_texts, 
                 tokenizer, slot_map, max_len=max_len):
    encoded_slots = np.zeros(shape=(len(all_texts), max_len), dtype=np.int32)
    
    for idx, text in enumerate(all_texts):
        enc = [] # for this idx, to be added at the end to encoded_slots
        
        # slot names for this idx
        slot_names = all_slots[idx]
        
        # raw word tokens
        # not using bert for this block because bert uses
        # a wordpiece tokenizer which will make 
        # the slot label to word mapping
        # difficult
        raw_tokens = text.split()

        # words or slot_values associated with a certain
        # slot_name are contained in the values of the
        # dict slots_names
        # now this becomes a two way lookup
        # first we check if a word belongs to any
        # slot label or not and then we add the value from
        # slot map to encoded for that word
        for rt in raw_tokens:
            # use bert tokenizer
            # to get wordpiece tokens
            bert_tokens = tokenizer.tokenize(rt)
            
            # find the slot name for a token
            rt_slot_name = get_slot_from_word(rt, text, slot_names)
            if rt_slot_name is not None:
                # fill with the slot_map value for all ber tokens for rt
                enc.append(slot_map[rt_slot_name])
                enc.extend([slot_map[rt_slot_name]] * (len(bert_tokens) - 1))

            else:
                # rt is not associated with any slot name
                enc.append(0)

        
        # now add to encoded_slots
        # ignore the first and the last elements
        # in encoded text as they're special chars
        encoded_slots[idx, 1:len(enc)+1] = enc
    
    return encoded_slots
    

In [ ]:
encoded_slots = encode_slots(slot_list, texts, tokenizer, slot_map)

In [ ]:
encoded_slots[0]

array([  0, 120, 120, 120, 120, 120,  46,  46,  46,  46, 120,  75,  75,
        64, 112,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

# Model

In [ ]:
from transformers import TFBertModel
from keras.layers import Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy

class JointIntentAndSlotFillingModel(tf.keras.Model):

    def __init__(self, intent_num_labels=None, slot_num_labels=None,
                 model_name=model_name, dropout_prob=0.1):
        super().__init__(name="joint_intent_slot")
        self.bert = TFBertModel.from_pretrained(model_name)
        self.dropout = Dropout(dropout_prob)
        self.intent_classifier = Dense(intent_num_labels,
                                       name="intent_classifier")
        self.slot_classifier = Dense(slot_num_labels,
                                     name="slot_classifier")
    def call(self, inputs, **kwargs):
        # two outputs from BERT
        trained_bert = self.bert.bert(inputs, **kwargs)
        pooled_output = trained_bert.pooler_output
        sequence_output = trained_bert.last_hidden_state
        
        # sequence_output will be used for slot_filling / classification
        sequence_output = self.dropout(sequence_output,
                                       training=kwargs.get("training", False))
        slot_logits = self.slot_classifier(sequence_output)

        # pooled_output for intent classification
        pooled_output = self.dropout(pooled_output,
                                     training=kwargs.get("training", False))
        intent_logits = self.intent_classifier(pooled_output)

        return slot_logits, intent_logits


In [ ]:
joint_model = JointIntentAndSlotFillingModel(
    intent_num_labels=len(intent_map), slot_num_labels=len(slot_map))

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
opt = Adam(learning_rate=3e-5, epsilon=1e-08)

# two outputs, one for slots, another for intents
# we have to fine tune for both
losses = [SparseCategoricalCrossentropy(from_logits=True),
          SparseCategoricalCrossentropy(from_logits=True)]

metrics = [SparseCategoricalAccuracy("accuracy")]
# compile model
joint_model.compile(optimizer=opt, loss=losses, metrics=metrics)

# First Pass

In [ ]:
x = {"input_ids": encoded_texts["input_ids"], "token_type_ids": encoded_texts["token_type_ids"],  "attention_mask": encoded_texts["attention_mask"]}

history = joint_model.fit(
    x, (encoded_slots, encoded_intents), epochs=1, batch_size=16, shuffle=True)

280/280 [==============================] - 237s 845ms/step - loss: 0.0052 - output_1_loss: 0.0032 - output_2_loss: 0.0020 - output_1_accuracy: 0.9992 - output_2_accuracy: 0.9993


In [ ]:
def nlu(text, tokenizer, model, intent_names, slot_names, max_token_length):
    # inputs = tf.constant(tokenizer.encode(text, create_O_slot(text)))[None, :]  # batch_size = 1
    # inputs = tf.constant(tokenizer.encode(text))[None, :]
    encoded_texts = tokenizer(text, padding= "max_length",truncation=True, max_length = max_token_length, return_tensors="tf")
    x = {"input_ids": encoded_texts["input_ids"], "token_type_ids": encoded_texts["token_type_ids"],  "attention_mask": encoded_texts["attention_mask"]}
    inputs = x
    outputs = model(inputs)
    slot_logits, intent_logits = outputs

    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, :]
    intent_id = intent_logits.numpy().argmax(axis=-1)[0]
    info = {"intent": intent_names[intent_id], "slots": {}}
    out_dict = {}
    # get all slot names and add to out_dict as keys
    predicted_slots = set([slot_names[s] for s in slot_ids if s != 0])
    for ps in predicted_slots:
      out_dict[ps] = []

    # check if the text starts with a small letter
    if text[0].islower():
      tokens = tokenizer.tokenize(text, add_special_tokens=True)
    else:
      tokens = tokenizer.tokenize(text)
    slot_list = []
    for token, slot_id in zip(tokens, slot_ids):
        slot_name = slot_names[slot_id]
        if slot_name == "<PAD>":
            continue
        if token == "[CLS]":
            continue
        if token == "[SEP]":
            continue

        # collect tokens
        collected_tokens = [token]
        idx = tokens.index(token)

        # see if it starts with ##
        # then it belongs to the previous token
        if token.startswith("##"):
          # check if the token already exists or not
          if tokens[idx - 1] not in out_dict[slot_name]:
            collected_tokens.insert(0, tokens[idx - 1])
        else:
          slot_list.append(slot_name)
        # add collected tokens to slots
        out_dict[slot_name].extend(collected_tokens)

    # process out_dict
    for slot_name in out_dict:
        tokens = out_dict[slot_name]
        slot_value = tokenizer.convert_tokens_to_string(tokens)

        info["slots"][slot_name] = slot_value.strip()
    info["slot list"] = slot_list

    return info


Sesuaikan path di bawah sesuai kebutuhan.

In [ ]:
joint_model.save("path/to/Joint Bert/Atis/first_pass_epoch_X_weight")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Manusia Learning/Joint Bert/Atis/first_pass_epoch_10_weight/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Manusia Learning/Joint Bert/Atis/first_pass_epoch_10_weight/assets


In [ ]:
df_atis_test.data[12]

In [ ]:
df_atis_test.encoding[12]

'O O O O B-fromloc.city_name I-fromloc.city_name O B-toloc.city_name O B-depart_date.date_relative B-depart_date.day_name O O B-toloc.city_name B-arrive_time.time_relative B-arrive_time.time I-arrive_time.time'

In [ ]:
nlu(df_atis_test.data[12], tokenizer, joint_model, intent_names, slot_names)

{'intent': 'atis_flight',
 'slot list': ['O',
  'O',
  'O',
  'O',
  'B-fromloc.city_name',
  'I-fromloc.city_name',
  'O',
  'B-toloc.city_name',
  'O',
  'B-depart_date.date_relative',
  'B-depart_date.day_name',
  'O',
  'O',
  'B-toloc.city_name',
  'B-arrive_time.time_relative',
  'B-arrive_time.time',
  'I-arrive_time.time'],
 'slots': {'B-arrive_time.time': '7',
  'B-arrive_time.time_relative': 'by',
  'B-depart_date.date_relative': 'next',
  'B-depart_date.day_name': 'wednesday',
  'B-fromloc.city_name': 'kansas',
  'B-toloc.city_name': 'chicago chicago',
  'I-arrive_time.time': 'pm',
  'I-fromloc.city_name': 'city',
  'O': 'show flight and prices to on arriving in'}}

# Evaluate First Pass

In [ ]:
pred_slot_first_pass = []
pred_intent_first_pass = []
for i in df_atis_test.values:
  info = nlu(i[0], tokenizer, joint_model, intent_names, slot_names)
  pred_slot_first_pass.append(info["slot list"])
  pred_intent_first_pass.append(info["intent"])

In [ ]:
test_slot = [i.split(" ") for i in df_atis_test.encoding]

In [ ]:
test_intent = df_atis_test.ground_truth.values

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(test_slot)
test_bin = mlb.transform(test_slot)
pred_bin = mlb.transform(pred_slot_first_pass)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) ['I-depart_date.today_relative'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_bin, pred_bin, target_names=mlb.classes_))

                              precision    recall  f1-score   support

             B-aircraft_code       1.00      0.94      0.97        33
              B-airline_code       0.79      0.70      0.74        33
              B-airline_name       0.99      0.99      0.99        95
              B-airport_code       1.00      0.22      0.36         9
              B-airport_name       0.89      0.38      0.53        21
 B-arrive_date.date_relative       1.00      1.00      1.00         2
      B-arrive_date.day_name       0.73      1.00      0.85        11
    B-arrive_date.day_number       0.83      0.83      0.83         6
    B-arrive_date.month_name       0.71      0.83      0.77         6
      B-arrive_time.end_time       0.89      1.00      0.94         8
 B-arrive_time.period_of_day       0.75      1.00      0.86         6
    B-arrive_time.start_time       0.88      0.88      0.88         8
          B-arrive_time.time       0.92      0.97      0.94        34
 B-arrive_time.time

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_intent, pred_intent_first_pass))

                             precision    recall  f1-score   support

          atis_abbreviation       0.94      0.97      0.96        33
              atis_aircraft       1.00      1.00      1.00         9
               atis_airfare       0.98      1.00      0.99        48
   atis_airfare#atis_flight       0.00      0.00      0.00         1
               atis_airline       1.00      1.00      1.00        38
               atis_airport       1.00      1.00      1.00        18
              atis_capacity       1.00      1.00      1.00        21
                  atis_city       1.00      0.67      0.80         6
              atis_day_name       0.00      0.00      0.00         2
              atis_distance       1.00      1.00      1.00        10
                atis_flight       0.98      0.99      0.99       632
   atis_flight#atis_airfare       0.83      0.42      0.56        12
   atis_flight#atis_airline       0.00      0.00      0.00         1
             atis_flight_no      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

In [ ]:
!pip install nervaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
new_slot_names = set()
for i in slot_names:
  if (i[0] == "B" or i[0] == "I"):
    new_slot_names.add(i[2:])
  else:
    new_slot_names.add(i)
# new_slot_names

In [ ]:
from nervaluate import Evaluator

res, res_type = Evaluator(test_slot, pred_slot_first_pass, tags=new_slot_names, loader="list").evaluate()

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,2474.000000,2516.000000,2377.000000,2516.000000
incorrect,221.000000,0.000000,318.000000,179.000000
partial,0.000000,179.000000,0.000000,0.000000
missed,138.000000,138.000000,138.000000,138.000000
spurious,239.000000,239.000000,239.000000,239.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2934.000000,2934.000000,2934.000000,2934.000000
precision,0.843217,0.888037,0.810157,0.857532
recall,0.873279,0.919696,0.839040,0.888104
f1,0.857985,0.903589,0.824345,0.872551


# Create B-only Slot

In [ ]:
first_slot = []
for i in df_atis_train.values:
  info = nlu(i[0], tokenizer, joint_model, intent_names, slot_names)
  first_slot.append(info["slot list"])

In [ ]:
def only_B_slot(text):
  temp = ""
  for i in text:
    if (i[0] == 'B'):
      temp += (" " + i)
    else:
      temp += " O"
  return temp.strip()

In [ ]:
second_slot = []
for i in first_slot:
  second_slot.append(only_B_slot(i))

In [ ]:
tds = encode_texts(tokenizer, texts, second_slot, max_token_length, "max_length")
tds.keys()
encoded_texts = tds

In [ ]:
{"input_ids": encoded_texts["input_ids"], "token_type_ids": encoded_texts["token_type_ids"],  "attention_mask": encoded_texts["attention_mask"]}

{'attention_mask': <tf.Tensor: shape=(4478, 252), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_ids': <tf.Tensor: shape=(4478, 252), dtype=int32, numpy=
 array([[ 101,  178, 1328, ...,    0,    0,    0],
        [ 101, 1668, 3868, ...,    0,    0,    0],
        [ 101, 1437, 1143, ...,    0,    0,    0],
        ...,
        [ 101, 1587, 1143, ...,    0,    0,    0],
        [ 101,  178,  112, ...,    0,    0,    0],
        [ 101, 1110, 1175, ...,    0,    0,    0]], dtype=int32)>,
 'token_type_ids': <tf.Tensor: shape=(4478, 252), dtype=int32, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>}

# Second Pass


In [ ]:
x = {"input_ids": encoded_texts["input_ids"], "token_type_ids": encoded_texts["token_type_ids"],  "attention_mask": encoded_texts["attention_mask"]}

history = joint_model.fit(
    x, (encoded_slots, encoded_intents), epochs=1, batch_size=16, shuffle=True)

280/280 [==============================] - 242s 864ms/step - loss: 0.0064 - output_1_loss: 0.0025 - output_2_loss: 0.0040 - output_1_accuracy: 0.9994 - output_2_accuracy: 0.9996


In [ ]:
df_atis_test.data[123]

'show me flight us 1500 on monday from charlotte to minneapolis please'

In [ ]:
df_atis_test.encoding[123]

'O O O B-airline_code B-flight_number O B-depart_date.day_name O B-fromloc.city_name O B-toloc.city_name O'

In [ ]:
nlu(df_atis_test.data[123], tokenizer, joint_model, intent_names, slot_names)

{'intent': 'atis_flight',
 'slot list': ['O',
  'O',
  'O',
  'B-airline_code',
  'B-flight_number',
  'O',
  'B-depart_date.day_name',
  'O',
  'B-fromloc.city_name',
  'O',
  'B-toloc.city_name',
  'O'],
 'slots': {'B-airline_code': 'us',
  'B-depart_date.day_name': 'monday',
  'B-flight_number': '1500',
  'B-fromloc.city_name': 'charlotte',
  'B-toloc.city_name': 'minneapolis',
  'O': 'show me flight on from to please'}}

In [ ]:
pred_slot_second_pass = []
pred_intent_second_pass = []
for i in df_atis_test.values:
  info = nlu(i[0], tokenizer, joint_model, intent_names, slot_names)
  pred_slot_second_pass.append(info["slot list"])
  pred_intent_second_pass.append(info["intent"])

# Evaluate Second Pass

In [ ]:
test_slot = [i.split(" ") for i in df_atis_test.encoding]

In [ ]:
test_intent = df_atis_test.ground_truth.values

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(test_slot)
test_bin = mlb.transform(test_slot)
pred_bin = mlb.transform(pred_slot_second_pass)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) ['I-depart_date.today_relative'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_bin, pred_bin, target_names=mlb.classes_))

                              precision    recall  f1-score   support

             B-aircraft_code       1.00      0.94      0.97        33
              B-airline_code       0.79      0.67      0.72        33
              B-airline_name       1.00      0.98      0.99        95
              B-airport_code       0.67      0.22      0.33         9
              B-airport_name       0.69      0.43      0.53        21
 B-arrive_date.date_relative       1.00      1.00      1.00         2
      B-arrive_date.day_name       0.73      1.00      0.85        11
    B-arrive_date.day_number       0.83      0.83      0.83         6
    B-arrive_date.month_name       0.71      0.83      0.77         6
      B-arrive_time.end_time       0.89      1.00      0.94         8
 B-arrive_time.period_of_day       0.75      1.00      0.86         6
    B-arrive_time.start_time       0.89      1.00      0.94         8
          B-arrive_time.time       0.94      0.97      0.96        34
 B-arrive_time.time

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_intent, pred_intent_second_pass))

                             precision    recall  f1-score   support

          atis_abbreviation       0.94      0.97      0.96        33
              atis_aircraft       1.00      1.00      1.00         9
               atis_airfare       0.98      1.00      0.99        48
   atis_airfare#atis_flight       0.00      0.00      0.00         1
               atis_airline       1.00      1.00      1.00        38
               atis_airport       1.00      1.00      1.00        18
              atis_capacity       1.00      1.00      1.00        21
                  atis_city       1.00      0.67      0.80         6
              atis_day_name       0.00      0.00      0.00         2
              atis_distance       1.00      1.00      1.00        10
                atis_flight       0.98      0.99      0.99       632
   atis_flight#atis_airfare       0.83      0.42      0.56        12
   atis_flight#atis_airline       0.00      0.00      0.00         1
             atis_flight_no      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

Sesuaikan path ini sesuai kebutuhan

In [ ]:
joint_model.save("path/to/Joint Bert/Atis/second_pass_epoch_X_weight")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Manusia Learning/Joint Bert/Atis/second_pass_epoch_10_weight/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Manusia Learning/Joint Bert/Atis/second_pass_epoch_10_weight/assets


In [ ]:
from nervaluate import Evaluator

res, res_type = Evaluator(test_slot, pred_slot_second_pass, tags=new_slot_names, loader="list").evaluate()

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,2494.000000,2529.000000,2399.000000,2529.000000
incorrect,201.000000,0.000000,296.000000,166.000000
partial,0.000000,166.000000,0.000000,0.000000
missed,138.000000,138.000000,138.000000,138.000000
spurious,231.000000,231.000000,231.000000,231.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2926.000000,2926.000000,2926.000000,2926.000000
precision,0.852358,0.892686,0.819891,0.864320
recall,0.880339,0.921991,0.846806,0.892693
f1,0.866123,0.907102,0.833131,0.878277


# Ringkasan evaluasi setiap part dan epoch

## ATIS

First pass epoch 2

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,2128.000000,2242.000000,1979.000000,2242.000000
incorrect,392.000000,0.000000,541.000000,278.000000
partial,0.000000,278.000000,0.000000,0.000000
missed,313.000000,313.000000,313.000000,313.000000
spurious,293.000000,293.000000,293.000000,293.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2813.000000,2813.000000,2813.000000,2813.000000
precision,0.756488,0.846427,0.703519,0.797014
recall,0.751147,0.840452,0.698553,0.791387
f1,0.753808,0.843429,0.701027,0.794191


Second pass epoch 2

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,2168.000000,2305.000000,2049.000000,2305.000000
incorrect,384.000000,0.000000,503.000000,247.000000
partial,0.000000,247.000000,0.000000,0.000000
missed,281.000000,281.000000,281.000000,281.000000
spurious,297.000000,297.000000,297.000000,297.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2849.000000,2849.000000,2849.000000,2849.000000
precision,0.760969,0.852404,0.719200,0.809056
recall,0.765267,0.857218,0.723262,0.813625
f1,0.763112,0.854805,0.721225,0.811334


First pass epoch 3

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,2275.000000,2376.000000,2140.000000,2376.000000
incorrect,336.000000,0.000000,471.000000,235.000000
partial,0.000000,235.000000,0.000000,0.000000
missed,222.000000,222.000000,222.000000,222.000000
spurious,283.000000,283.000000,283.000000,283.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2894.000000,2894.000000,2894.000000,2894.000000
precision,0.786109,0.861610,0.739461,0.821009
recall,0.803036,0.880162,0.755383,0.838687
f1,0.794482,0.870787,0.747337,0.829754


Second pass epoch 3

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,2318.000000,2420.000000,2199.000000,2420.000000
incorrect,315.000000,0.000000,434.000000,213.000000
partial,0.000000,213.000000,0.000000,0.000000
missed,200.000000,200.000000,200.000000,200.000000
spurious,254.000000,254.000000,254.000000,254.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2887.000000,2887.000000,2887.000000,2887.000000
precision,0.802910,0.875130,0.761690,0.838240
recall,0.818214,0.891811,0.776209,0.854218
f1,0.810490,0.883392,0.768881,0.846154


First pass epoch 4

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,2364.000000,2450.000000,2243.000000,2450.000000
incorrect,301.000000,0.000000,422.000000,215.000000
partial,0.000000,215.000000,0.000000,0.000000
missed,168.000000,168.000000,168.000000,168.000000
spurious,259.000000,259.000000,259.000000,259.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2924.000000,2924.000000,2924.000000,2924.000000
precision,0.808482,0.874658,0.767100,0.837893
recall,0.834451,0.902753,0.791740,0.864808
f1,0.821261,0.888484,0.779225,0.851138


Second pass epoch 4

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,2389.000000,2469.000000,2278.000000,2469.000000
incorrect,282.000000,0.000000,393.000000,202.000000
partial,0.000000,202.000000,0.000000,0.000000
missed,162.000000,162.000000,162.000000,162.000000
spurious,261.000000,261.000000,261.000000,261.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2932.000000,2932.000000,2932.000000,2932.000000
precision,0.814802,0.876535,0.776944,0.842087
recall,0.843276,0.907166,0.804095,0.871514
f1,0.828794,0.891587,0.790286,0.856548


First pass epoch 5

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,2422.000000,2470.000000,2311.000000,2470.000000
incorrect,249.000000,0.000000,360.000000,201.000000
partial,0.000000,201.000000,0.000000,0.000000
missed,162.000000,162.000000,162.000000,162.000000
spurious,244.000000,244.000000,244.000000,244.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2915.000000,2915.000000,2915.000000,2915.000000
precision,0.830875,0.881818,0.792796,0.847341
recall,0.854924,0.907342,0.815743,0.871867
f1,0.842728,0.894398,0.804106,0.859429


,ent_type,partial,strict,exact
correct,2422.000000,2470.000000,2311.000000,2470.000000
incorrect,249.000000,0.000000,360.000000,201.000000
partial,0.000000,201.000000,0.000000,0.000000
missed,162.000000,162.000000,162.000000,162.000000
spurious,244.000000,244.000000,244.000000,244.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2915.000000,2915.000000,2915.000000,2915.000000
precision,0.830875,0.881818,0.792796,0.847341
recall,0.854924,0.907342,0.815743,0.871867
f1,0.842728,0.894398,0.804106,0.859429


Second pass epoch 5

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,2445.000000,2500.000000,2345.000000,2500.000000
incorrect,238.000000,0.000000,338.000000,183.000000
partial,0.000000,183.000000,0.000000,0.000000
missed,150.000000,150.000000,150.000000,150.000000
spurious,236.000000,236.000000,236.000000,236.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2919.000000,2919.000000,2919.000000,2919.000000
precision,0.837616,0.887804,0.803357,0.856458
recall,0.863043,0.914755,0.827744,0.882457
f1,0.850139,0.901078,0.815369,0.869263


First pass epoch 6

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,2443.000000,2491.000000,2334.000000,2491.000000
incorrect,238.000000,0.000000,347.000000,190.000000
partial,0.000000,190.000000,0.000000,0.000000
missed,152.000000,152.000000,152.000000,152.000000
spurious,257.000000,257.000000,257.000000,257.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2938.000000,2938.000000,2938.000000,2938.000000
precision,0.831518,0.880191,0.794418,0.847856
recall,0.862337,0.912813,0.823862,0.879280
f1,0.846647,0.896205,0.808872,0.863282


Second pass epoch 6

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,2467.000000,2512.000000,2367.000000,2512.000000
incorrect,220.000000,0.000000,320.000000,175.000000
partial,0.000000,175.000000,0.000000,0.000000
missed,146.000000,146.000000,146.000000,146.000000
spurious,233.000000,233.000000,233.000000,233.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2920.000000,2920.000000,2920.000000,2920.000000
precision,0.844863,0.890240,0.810616,0.860274
recall,0.870808,0.917579,0.835510,0.886693
f1,0.857639,0.903702,0.822875,0.873284


First pass epoch 7

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,2488.000000,2526.000000,2389.000000,2526.000000
incorrect,211.000000,0.000000,310.000000,173.000000
partial,0.000000,173.000000,0.000000,0.000000
missed,134.000000,134.000000,134.000000,134.000000
spurious,236.000000,236.000000,236.000000,236.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2935.000000,2935.000000,2935.000000,2935.000000
precision,0.847700,0.890119,0.813969,0.860647
recall,0.878221,0.922167,0.843276,0.891634
f1,0.862691,0.905860,0.828363,0.875867


Second pass epoch 7

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,2482.000000,2494.000000,2374.000000,2494.000000
incorrect,203.000000,0.000000,311.000000,191.000000
partial,0.000000,191.000000,0.000000,0.000000
missed,148.000000,148.000000,148.000000,148.000000
spurious,238.000000,238.000000,238.000000,238.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2923.000000,2923.000000,2923.000000,2923.000000
precision,0.849128,0.885905,0.812179,0.853233
recall,0.876103,0.914049,0.837981,0.880339
f1,0.862404,0.899757,0.824878,0.866574


First pass epoch 8

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,2485.000000,2521.000000,2389.000000,2521.000000
incorrect,212.000000,0.000000,308.000000,176.000000
partial,0.000000,176.000000,0.000000,0.000000
missed,136.000000,136.000000,136.000000,136.000000
spurious,231.000000,231.000000,231.000000,231.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2928.000000,2928.000000,2928.000000,2928.000000
precision,0.848702,0.891052,0.815915,0.860997
recall,0.877162,0.920932,0.843276,0.889869
f1,0.862697,0.905746,0.829370,0.875195


Second pass epoch 8

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,2477.000000,2524.000000,2388.000000,2524.000000
incorrect,213.000000,0.000000,302.000000,166.000000
partial,0.000000,166.000000,0.000000,0.000000
missed,143.000000,143.000000,143.000000,143.000000
spurious,234.000000,234.000000,234.000000,234.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2924.000000,2924.000000,2924.000000,2924.000000
precision,0.847127,0.891587,0.816689,0.863201
recall,0.874338,0.920226,0.842923,0.890928
f1,0.860518,0.905680,0.829599,0.876846


First pass epoch 10

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,2474.000000,2516.000000,2377.000000,2516.000000
incorrect,221.000000,0.000000,318.000000,179.000000
partial,0.000000,179.000000,0.000000,0.000000
missed,138.000000,138.000000,138.000000,138.000000
spurious,239.000000,239.000000,239.000000,239.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2934.000000,2934.000000,2934.000000,2934.000000
precision,0.843217,0.888037,0.810157,0.857532
recall,0.873279,0.919696,0.839040,0.888104
f1,0.857985,0.903589,0.824345,0.872551


Second pass epoch 10

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,2494.000000,2529.000000,2399.000000,2529.000000
incorrect,201.000000,0.000000,296.000000,166.000000
partial,0.000000,166.000000,0.000000,0.000000
missed,138.000000,138.000000,138.000000,138.000000
spurious,231.000000,231.000000,231.000000,231.000000
possible,2833.000000,2833.000000,2833.000000,2833.000000
actual,2926.000000,2926.000000,2926.000000,2926.000000
precision,0.852358,0.892686,0.819891,0.864320
recall,0.880339,0.921991,0.846806,0.892693
f1,0.866123,0.907102,0.833131,0.878277


# Load model

Untuk mencoba menjalankan model, diharuskan untuk menjalankan bagian *Installs*, *Imports*, *Tokenize*, dan *Mapping slots and intents*.

In [ ]:
def nlu(text, tokenizer, model, intent_names, slot_names, max_token_length):
    # inputs = tf.constant(tokenizer.encode(text, create_O_slot(text)))[None, :]  # batch_size = 1
    # inputs = tf.constant(tokenizer.encode(text))[None, :]
    encoded_texts = tokenizer(text, padding= "max_length",truncation=True, max_length = max_token_length, return_tensors="tf")
    x = {"input_ids": encoded_texts["input_ids"], "token_type_ids": encoded_texts["token_type_ids"],  "attention_mask": encoded_texts["attention_mask"]}
    inputs = x
    outputs = model(inputs)
    slot_logits, intent_logits = outputs

    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, :]
    intent_id = intent_logits.numpy().argmax(axis=-1)[0]
    info = {"intent": intent_names[intent_id], "slots": {}}
    out_dict = {}
    # get all slot names and add to out_dict as keys
    predicted_slots = set([slot_names[s] for s in slot_ids if s != 0])
    for ps in predicted_slots:
      out_dict[ps] = []

    # check if the text starts with a small letter
    if text[0].islower():
      tokens = tokenizer.tokenize(text, add_special_tokens=True)
    else:
      tokens = tokenizer.tokenize(text)
    slot_list = []
    for token, slot_id in zip(tokens, slot_ids):
        slot_name = slot_names[slot_id]
        if slot_name == "<PAD>":
            continue
        if token == "[CLS]":
            continue
        if token == "[SEP]":
            continue

        # collect tokens
        collected_tokens = [token]
        idx = tokens.index(token)

        # see if it starts with ##
        # then it belongs to the previous token
        if token.startswith("##"):
          # check if the token already exists or not
          if tokens[idx - 1] not in out_dict[slot_name]:
            collected_tokens.insert(0, tokens[idx - 1])
        else:
          slot_list.append(slot_name)
        # add collected tokens to slots
        out_dict[slot_name].extend(collected_tokens)

    # process out_dict
    for slot_name in out_dict:
        tokens = out_dict[slot_name]
        slot_value = tokenizer.convert_tokens_to_string(tokens)

        info["slots"][slot_name] = slot_value.strip()
    info["slot list"] = slot_list

    return info


Sesuaikan path di bawah ini dengan path menuju folder yang sesuai (harus `second_pass_epoch_10_weight`). Silakan merujuk pada link https://drive.google.com/drive/folders/1vuf4IZYyoInyYdise5S9LZE9003Lb6sN untuk struktur folder.

In [ ]:
joint_model = tf.keras.models.load_model("path/to/Joint Bert/Atis/second_pass_epoch_10_weight", compile=True)

In [ ]:
nlu("after 12 pm on wednesday april sixth i would like to fly from manado to russia", tokenizer, joint_model, intent_names, slot_names, max_token_length)

{'intent': 'atis_flight',
 'slot list': ['B-depart_time.time_relative',
  'B-depart_time.time',
  'I-depart_time.time',
  'O',
  'B-depart_date.day_name',
  'B-depart_date.month_name',
  'B-depart_date.day_number',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-fromloc.city_name',
  'O',
  'B-toloc.city_name'],
 'slots': {'B-depart_date.day_name': 'wednesday',
  'B-depart_date.day_number': 'sixth',
  'B-depart_date.month_name': 'april',
  'B-depart_time.time': '12',
  'B-depart_time.time_relative': 'after',
  'B-fromloc.city_name': 'manado',
  'B-toloc.city_name': 'russia',
  'I-airline_name': '',
  'I-depart_time.time': 'pm',
  'O': 'on i would like to fly from to'}}

In [ ]:
pred_slot_second_pass = []
pred_intent_second_pass = []
for i in df_atis_test.values:
  info = nlu(i[0], tokenizer, joint_model, intent_names, slot_names, max_token_length)
  pred_slot_second_pass.append(info["slot list"])
  pred_intent_second_pass.append(info["intent"])

In [ ]:
pred_slot_second_pass[20:25]

[['O',
  'O',
  'O',
  'B-fromloc.city_name',
  'O',
  'B-toloc.city_name',
  'O',
  'B-depart_date.day_name'],
 ['O',
  'O',
  'B-depart_date.day_name',
  'B-depart_time.period_of_day',
  'O',
  'B-fromloc.city_name',
  'O',
  'B-toloc.city_name',
  'B-toloc.city_name',
  'I-toloc.city_name'],
 ['O',
  'O',
  'B-depart_date.day_name',
  'B-depart_time.period_of_day',
  'O',
  'B-fromloc.city_name',
  'B-fromloc.city_name',
  'I-fromloc.city_name',
  'O',
  'B-toloc.city_name'],
 ['O',
  'O',
  'O',
  'O',
  'B-fromloc.city_name',
  'I-fromloc.city_name',
  'O',
  'B-toloc.city_name',
  'I-toloc.city_name'],
 ['O',
  'O',
  'O',
  'O',
  'B-fromloc.city_name',
  'I-fromloc.city_name',
  'O',
  'B-toloc.city_name',
  'I-toloc.city_name']]

In [ ]:
pred_intent_second_pass[20:25]

['atis_flight', 'atis_flight', 'atis_flight', 'atis_flight', 'atis_flight']